In [59]:
import pandas as pd
from cassandra.cluster import Cluster

Create a session connection to Cassandra cluster

In [60]:
clstr = Cluster()
session = clstr.connect()

In [61]:
session.execute("DROP KEYSPACE IF EXISTS songs_db")

In [63]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Creating keyspace songs_db

In [64]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songs_db WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':2
    }
""")

In [65]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

songs_db
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Creating Table Users

In [66]:
session.execute("""
CREATE TABLE IF NOT EXISTS songs_db.users ( 
    SongID INT, 
    Title TEXT, 
    Duration FLOAT, 
    Genre Text,           
    PRIMARY KEY(SongID)
);
""")

Reading Data

In [67]:
df = pd.read_csv('exam.csv')
df

,SongID,Title,Duration,Genre,Unnamed: 4
0,1000,'Perfect',3.3,'Pop',NaN
1,1001,'Cool',2.1,'rock',NaN
2,1002,'Hot',3.0,'classical',NaN
3,1003,'Dope',4.0,'rock',NaN
4,1004,'Jim',2.1,'country',NaN
5,1005,'Frank',4.2,'rock',NaN
6,1006,'Judy',4.0,'classical',NaN
7,1007,'Tom',3.0,'country',NaN
8,1008,'Bill',3.5,'rock',NaN


In [68]:
for index, row in df.iterrows():
    print(f"SongID = {row.SongID}, Title ={row.Title}, Duration = {row.Duration}, Genre = {row.Genre}")

SongID = 1000, Title ='Perfect', Duration = 3.3, Genre = 'Pop'
SongID = 1001, Title ='Cool', Duration = 2.1, Genre = 'rock'
SongID = 1002, Title ='Hot', Duration = 3.0, Genre = 'classical'
SongID = 1003, Title ='Dope', Duration = 4.0, Genre = 'rock'
SongID = 1004, Title ='Jim', Duration = 2.1, Genre = 'country'
SongID = 1005, Title ='Frank', Duration = 4.2, Genre = 'rock'
SongID = 1006, Title ='Judy', Duration = 4.0, Genre = 'classical'
SongID = 1007, Title ='Tom', Duration = 3.0, Genre = 'country'
SongID = 1008, Title ='Bill', Duration = 3.5, Genre = 'rock'


Inserting into table

In [70]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO songs_db.users (SongID, Title, Duration, Genre)     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )
    session.execute(f"""
        INSERT INTO songs_db.users (SongID, Title, Duration, Genre))     
        VALUES ({row.SongID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )


        INSERT INTO songs_db.users (SongID, Title, Duration, Genre)     
        VALUES (1000, 'Perfect', 3.3, 'Pop');
        


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 2:67 mismatched input ')' expecting K_VALUES (..., Title, Duration, Genre)[)]...)">

In [73]:

insert_query = """
INSERT INTO songs_db.users (SongID, Title, Duration, Genre)
VALUES (?, ?, ?, ?)
"""

prepared_insert = session.prepare(insert_query)


data_to_insert = [
    (123, 'Pdgsaghace', 345, 'pop'),
    (124, 'Ccbhsagcjqn', 201, 'rock'),
    (125, 'xfasghcbq', 236.5, 'classical'),
    (126, 'Dontcshvajc', 129, 'country'),
    (127, 'cvsgahjc', 236, 'pop'),
    (128, 'csaghvcja', 212.5, 'rock')
]
# Insert each row in a loop
for row in data_to_insert:
    session.execute(prepared_insert, row)

Using select statement

In [74]:
rows = session.execute("select (SongID, Title, Duration, Genre) from songs_db.users ")
for row in rows:
    print(f"SongID={row[0][0]}, Title={row[0][1]}, Duration={row[0][2]}, Genre={row[0][3]}")

SongID=128, Title=csaghvcja, Duration=212.5, Genre=rock
SongID=123, Title=Pdgsaghace, Duration=345.0, Genre=pop
SongID=125, Title=xfasghcbq, Duration=236.5, Genre=classical
SongID=126, Title=Dontcshvajc, Duration=129.0, Genre=country
SongID=127, Title=cvsgahjc, Duration=236.0, Genre=pop
SongID=124, Title=Ccbhsagcjqn, Duration=201.0, Genre=rock


Using update

In [78]:
rows = session.execute("update songs_db.users set Title = 'flop' where SongId = 123 ") 
for row in rows:
    print(f"SongID={row[0][0]}, Title={row[0][1]}, Duration={row[0][2]}, Genre={row[0][3]}")

Using delete

In [81]:
rows = session.execute("Delete from songs_db.users  where SongId = 125  ")
for row in rows:
    print(row)